# Ipswich v. Newcastle United - 2024-12-21 match analysis
*This notebook intends to walk through the theory and calculations I am currently using in my model to calculate probability of 1 x 2 result and a table of over / under goal projections. I will break the calculations out into utility functions stored in `../src/utils` for ease of use in future explorations. This is to document the calculations.*

### Instructions:

1. Ensure all cells in all notebooks in `/notebooks` directory have been run.
2. Define `home_side` and `away_side` in first cell. If making a comparison with odds from `03_retrieve_odds.ipynb` also enter matchday.
3. Click 'run all'
4. Probability for 1 x 2 and over / under can be viewed in the last cell
5. *Suggestion:* copy and paste notebook to `../output/< matchday_directory >/` and rename file. IMPORTANT: Update file path in lines 14 and 15 if needed if you make a copy.



### Import League Data & Define Sides

Define 'home_side' and 'away_side'
Ensure correct output



In [3]:
import pandas as pd
from scipy.stats import poisson
import numpy as np

"""
BUG: if copying from `notebooks/03_analysis_template_notebook.ipynb` to a directory in output
for storage, ensure the file path for reading the csv will need to be updated from '../data/processed/home_table.csv'
to '../../data/processed/home_table.csv' if frames are run again.

TLDR
if you are in ./notebooks/03_analysis_tempalte.csv you may run this cell as is. If you are not, check that following
read_csv commands read from the correct directory.
"""
home_dataframe = pd.read_csv('../data/processed/home_table.csv')
away_dataframe = pd.read_csv('../data/processed/away_table.csv')

"""
!!! INPUT TEAMS & MATCHDAY HERE !!!

wrapped in quotes, eg.

home_side = "Bournemouth"
away_side = "West Ham"
matchday = "2024-12-16" # (optional)

If there is a ValueError,
(1) check spelling against the error message. Must be exact.
(2) check capitalization
(3) check that it is wrapped in quotes

!!! INPUT TEAMS & MATCHDAY HERE !!!
"""
home_side = "Ipswich"
away_side = "Newcastle United"
matchday = "2024-12-21" # (optional)

# validate input
clubs = set(home_dataframe['Team'].values)
if home_side not in clubs:
    raise ValueError(f"Invalid entry for 'home_side'. Expected one of {clubs}, but received '{home_side}'")
elif away_side not in clubs:
    raise ValueError(f"Invalid entry for 'away_side'. Expected one of {clubs}, but received '{away_side}'")
else:
    print(f"Ready to calculate probability for {home_side} v. {away_side}")

Ready to calculate probability for Ipswich v. Newcastle United


### Calculate Projected Goals

In [4]:
# 2. grab their gpm_scored and gpm_conceded
home_side_gpm_scored = home_dataframe.loc[
    home_dataframe["Team"] == home_side,
    "gpm_scored"
].values[0]

home_side_gpm_conceded = home_dataframe.loc[
    home_dataframe["Team"] == home_side,
    "gpm_conceded"
].values[0]

print(home_side_gpm_scored, home_side_gpm_conceded)

away_side_gpm_scored = away_dataframe.loc[
    away_dataframe["Team"] == away_side,
    "gpm_scored"
].values[0]

away_side_gpm_conceded = away_dataframe.loc[
    away_dataframe["Team"] == away_side,
    "gpm_conceded"
].values[0]

print(away_side_gpm_scored, away_side_gpm_conceded)

# 3.
epl_home_avg_gpm_scored = round(sum(home_dataframe["gpm_scored"].values) / 20, 2)
epl_home_avg_gpm_conceded = round(sum(home_dataframe["gpm_conceded"].values) / 20, 2)
print(epl_home_avg_gpm_scored, epl_home_avg_gpm_conceded)

# 4. create a ratio (attack and defense rating) of their gpm to the league average gpm
home_attack_rating = home_side_gpm_scored / epl_home_avg_gpm_scored
away_defense_rating = away_side_gpm_conceded / epl_home_avg_gpm_scored
away_attack_rating = away_side_gpm_scored / epl_home_avg_gpm_conceded
home_defense_rating = home_side_gpm_conceded / epl_home_avg_gpm_conceded

print(home_attack_rating, away_defense_rating, away_attack_rating, home_defense_rating)

# home projected goals
home_projected_goals = home_attack_rating * away_defense_rating * epl_home_avg_gpm_scored
# away projected goals
away_projected_goals = away_attack_rating * home_defense_rating * epl_home_avg_gpm_conceded
print("\n ================== projected_goals ==================== \n")
print(f"HOME {home_side} projected goals:", home_projected_goals)
print(f"AWAY {away_side} projected goals: ", away_projected_goals)

0.75 1.5
1.375 1.625
1.61 1.4
0.4658385093167702 1.0093167701863353 0.9821428571428572 1.0714285714285714

 ================== projected_goals ==================== 

HOME Ipswich projected goals: 0.7569875776397514
AWAY Newcastle United projected goals:  1.4732142857142858


### Poisson Distribution of Goal Probabilities

In [5]:
# Poisson probability for home team goals 0 - 8
home_score_prob = []
away_score_prob = []
for i in range(9):
    prob = poisson.pmf(i, home_projected_goals)
    home_score_prob.append(prob)
for i in range(9):
    prob = poisson.pmf(i, away_projected_goals)
    away_score_prob.append(prob)

home_score_prob = np.array(home_score_prob)
away_score_prob = np.array(away_score_prob)
# print(home_score_prob)
# print(away_score_prob)

exact_score_prob = np.outer(away_score_prob, home_score_prob)
print(f"{home_side} v. {away_side}", exact_score_prob)

Ipswich v. Newcastle United [[1.07506726e-01 8.13812563e-02 3.08023000e-02 7.77231950e-03
  1.47088733e-03 2.22688687e-04 2.80954283e-05 3.03827003e-06
  2.87491584e-07]
 [1.58380445e-01 1.19892029e-01 4.53783884e-02 1.14502921e-02
  2.16693222e-03 3.28068155e-04 4.13905863e-05 4.47602281e-06
  4.23536708e-07]
 [1.16664167e-01 8.83133252e-02 3.34260451e-02 8.43436696e-03
  1.59617775e-03 2.41657346e-04 3.04886015e-05 3.29707037e-06
  3.11980164e-07]
 [5.72904392e-02 4.33681508e-02 1.64145757e-02 4.14187663e-03
  7.83837290e-04 1.18671018e-04 1.49720811e-05 1.61909706e-06
  1.53204545e-07]
 [2.11002734e-02 1.59726448e-02 6.04554685e-03 1.52546796e-03
  2.88690073e-04 4.37069598e-05 5.51427094e-06 5.96319229e-07
  5.64257811e-08]
 [6.21704483e-03 4.70622570e-03 1.78127720e-03 4.49468237e-04
  8.50604680e-05 1.28779435e-05 1.62474055e-06 1.75701201e-07
  1.66254534e-08]
 [1.52650654e-03 1.15554649e-03 4.37367169e-04 1.10360505e-04
  2.08853828e-05 3.16199506e-06 3.98931830e-07 4.31409200e

### Under Market

In [6]:
thresholds = [0.5, 1.5, 2.5, 3.5]
sums = []

for t in thresholds:
    mask = np.add.outer(
        np.arange(
            exact_score_prob.shape[0]
        ),
        np.arange(
            exact_score_prob.shape[1]
        )
    ) <= t
    # print(mask)
    sums.append(exact_score_prob[mask].sum())

# print(sums)
under_market_df = pd.DataFrame({
    "Goals": thresholds,
    "Prob": sums
})
under_market_df["Implied Odds"] = 1 / under_market_df["Prob"]
print("~  ~ under market ~  ~\n", f"{home_side} v. {away_side}\n", under_market_df)

~  ~ under market ~  ~
 Ipswich v. Newcastle United
    Goals      Prob  Implied Odds
0    0.5  0.107507      9.301744
1    1.5  0.347268      2.879617
2    2.5  0.614627      1.627003
3    3.5  0.813381      1.229436


In [7]:
thresholds = [0.5, 1.5, 2.5, 3.5]
sums = []

for t in thresholds:
    mask = np.add.outer(
        np.arange(
            exact_score_prob.shape[0]
        ),
        np.arange(
            exact_score_prob.shape[1]
        )
    ) >= t
    sums.append(exact_score_prob[mask].sum())

# print(sums)
over_market_df = pd.DataFrame({
    "Goals": thresholds,
    "Prob": sums
})

over_market_df["Implied Odds"] = 1 / over_market_df["Prob"]
print("~  ~ over market ~  ~\n", f"{home_side} v. {away_side}\n", over_market_df)

~  ~ over market ~  ~
 Ipswich v. Newcastle United
    Goals      Prob  Implied Odds
0    0.5  0.892469      1.120487
1    1.5  0.652707      1.532080
2    2.5  0.385349      2.595051
3    3.5  0.186594      5.359219


### 1 x 2 Probability

In [8]:
# Poisson probability for home team goals 0 - 8
home_score_prob = []
away_score_prob = []
for i in range(9):
    prob = poisson.pmf(i, home_projected_goals)
    home_score_prob.append(prob)
for i in range(9):
    prob = poisson.pmf(i, away_projected_goals)
    away_score_prob.append(prob)

home_score_prob = np.array(home_score_prob)
away_score_prob = np.array(away_score_prob)
# print(home_score_prob)
# print(away_score_prob)

exact_score_prob = np.outer(away_score_prob, home_score_prob)

# print(exact_score_prob)
home_win_prob = np.sum(np.triu(exact_score_prob, k=1))
away_win_prob = np.sum(np.tril(exact_score_prob, k=-1))
draw_prob = np.trace(exact_score_prob)

# prepare data in readable dataframe
data = {
    "winner": [f"{home_side} (1)", "Draw (x)", f"{away_side} (2)"],
    "prob": [home_win_prob, draw_prob, away_win_prob],
}
result_df = pd.DataFrame(data)
result_df['implied_odds'] = 1 / result_df['prob']
print("      ~  ~ moneyline market ~  ~\n", result_df)

      ~  ~ moneyline market ~  ~
                  winner      prob  implied_odds
0           Ipswich (1)  0.192328      5.199447
1              Draw (x)  0.265269      3.769763
2  Newcastle United (2)  0.542379      1.843729


# Full Results Summary:

In [9]:
# helper function to translate discrepency between Understat and Odds-Api naming
def translate(string):

    translations = {
        "Brighton": "Brighton and Hove Albion",
        "Ipswich": "Ipswich Town",
        "Leicester": "Leicester City",
        "Tottenham": "Tottenham Hotspur",
        "West Ham": "West Ham United"
    }

    if string in translations:
        return translations[string]
    return string

home_side, away_side = translate(home_side), translate(away_side)

match_string = f"{home_side} v. {away_side}"

print(f"CALCULATED PROBABILITIES FOR {home_side} v. {away_side}")
print("\n\n        ~ ~ moneyline market ~ ~\n\n", result_df)
print("\n\n          ~ ~ over market ~ ~ \n\n", over_market_df)
print("\n\n          ~ ~ under market ~ ~ \n\n", under_market_df)

try:
    odds_df = pd.read_csv(f"../data/processed/odds/odds_matchday_{matchday}.csv")
    filtered_odds = odds_df.loc[odds_df["match"] == match_string]
    implied = {
        "1": 1 / filtered_odds["1"],
        "x": 1 / filtered_odds["x"],
        "2": 1 / filtered_odds["2"],
        "point": filtered_odds["ovr_und_point"],
        "over": 1 / filtered_odds["over"],
        "under": 1 / filtered_odds["under"]
    }
    implied_odds_df = pd.DataFrame(implied)

    print(f"\n ~~ MyBookie Odds for {match_string} ~~\n",filtered_odds.drop(columns=["match"]))
    print(f"\n ~~ MyBookie Odds IMPLIED for {match_string} ~~\n", implied_odds_df)
except:
    print(f"No data for sportsbook odds on matchday {matchday}. To view against real odds, run `notebooks/03_retrieve_odds.ipynb` ran correctly and ensure matchday is correct in the first cell of this notebook")


CALCULATED PROBABILITIES FOR Ipswich Town v. Newcastle United


        ~ ~ moneyline market ~ ~

                  winner      prob  implied_odds
0           Ipswich (1)  0.192328      5.199447
1              Draw (x)  0.265269      3.769763
2  Newcastle United (2)  0.542379      1.843729


          ~ ~ over market ~ ~ 

    Goals      Prob  Implied Odds
0    0.5  0.892469      1.120487
1    1.5  0.652707      1.532080
2    2.5  0.385349      2.595051
3    3.5  0.186594      5.359219


          ~ ~ under market ~ ~ 

    Goals      Prob  Implied Odds
0    0.5  0.107507      9.301744
1    1.5  0.347268      2.879617
2    2.5  0.614627      1.627003
3    3.5  0.813381      1.229436

 ~~ MyBookie Odds for Ipswich Town v. Newcastle United ~~
      1     x    2  ovr_und_point  over  under
3  4.5  3.85  1.7            2.5   1.6   2.25

 ~~ MyBookie Odds IMPLIED for Ipswich Town v. Newcastle United ~~
           1        x         2  point   over     under
3  0.222222  0.25974  0.588235   

# Conclusion and Pick
SAFE: I like Newcastle to win this, I don't love the moneyline offer here. Leaning towards no bet.
### My Pick:

**NO BET** 